<a href="https://colab.research.google.com/github/IDeal7/ML/blob/master/compilerlabeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from pathlib import Path
import os
import re
import numpy as np

def load_data():
    count = 0
    x_data=[]
    y_data=[]

    xAppend = x_data.append
    yAppend = y_data.append

    for p in Path("./Gather/").glob("**\\*."+"png"):
        p = str(p).replace("\\", "/")
        file = plt.imread(p).copy()

        file.resize((56, 56))

        xAppend(file)

        #label = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]  # 옵티마이즈별 라벨링
        # label[(int(p[10:11])-3) * 4 + int(p[13:14])] = 1

        label = [0, 0, 0, 0, 0, 0, 0]  # 컴파일러별 라벨링
        label[int(p[10:11])-3] = 1
        print(int(p[10:11]))
        yAppend(label)

        count += 1
        print(count)

    return x_data, y_data


def cnn():
    x_data, y_data = load_data()

    x_data = np.array(x_data)  # 462
    y_data = np.array(y_data)

    x_data = np.reshape(x_data,(x_data.shape + (1,)))
    #y_data = np.reshape(y_data,(y_data.shape + (7,)))
    #y_data = np.expand_dims(y_data,axis=)

    print(x_data.shape)
    print(y_data.shape)

    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2)
    batch_size = 512
    num_classes = 7
    epochs = 20

    # input image dimensions
    img_rows, img_cols = 56, 56

    '''
    if K.image_data_format() == 'channels_first':
        x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
        x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
        input_shape = (1, img_rows, img_cols)
    else:
        x_train = x_train.astype('float32')
        x_test = x_test.astype('float32')
    '''

    input_shape = (img_rows, img_cols,1)
#    x_test /= 255

    # convert class vectors to binary class matrices
#    y_train = keras.utils.to_categorical(y_train, num_classes, dtype='int')
#    y_test = keras.utils.to_categorical(y_test, num_classes, dtype='int')

    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))

    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

    model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

cnn()

ValueError: ignored